In [25]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from segrnn import SegRNNModel
from pytorch_lightning import Trainer

In [15]:
# Load the dataset
# Replace 'your_dataset.csv' with your actual data file
df = pd.read_csv('JRB.csv', low_memory=False)

# Step 1: Ensure 'valid' column is datetime and sort the DataFrame
df['valid'] = pd.to_datetime(df['valid'])
df = df.sort_values(by=['station', 'valid']).reset_index(drop=True)

In [16]:
df.head()

,station,valid,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,...,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
0,JRB,2016-07-21 08:15:00,73.40,66.20,78.19,240.00,5.00,0.00,30.17,M,...,M,M,M,M,M,M,M,73.40,KJRB 211215Z 24005KT 10SM CLR 23/19 A3017 RMK AO1,M
1,JRB,2016-07-21 08:35:00,73.40,66.20,78.19,240.00,5.00,0.00,30.17,M,...,M,M,M,M,M,M,M,73.40,KJRB 211235Z 24005KT 10SM CLR 23/19 A3017 RMK AO1,M
2,JRB,2016-07-21 08:55:00,75.20,66.20,73.61,240.00,6.00,0.00,30.17,M,...,M,M,M,M,M,M,M,75.20,KJRB 211255Z 24006KT 10SM CLR 24/19 A3017 RMK AO1,M
3,JRB,2016-07-21 09:15:00,75.20,66.20,73.61,240.00,5.00,0.00,30.17,M,...,M,M,M,M,M,M,M,75.20,KJRB 211315Z 24005KT 10SM CLR 24/19 A3017 RMK AO1,M
4,JRB,2016-07-21 09:35:00,78.80,66.20,65.33,240.00,5.00,0.00,30.16,M,...,M,M,M,M,M,M,M,80.82,KJRB 211335Z 24005KT 10SM CLR 26/19 A3016 RMK AO1,M


In [17]:
# Step 2: Replace placeholders with np.nan in continuous columns
continuous_cols = ['tmpf', 'dwpf', 'relh', 'feel', 'drct', 'sknt', 'gust',
                   'peak_wind_gust', 'peak_wind_drct', 'alti', 'mslp', 'vsby',
                   'p01i', 'ice_accretion_1hr', 'ice_accretion_3hr', 'ice_accretion_6hr',
                   'skyl1', 'skyl2', 'skyl3', 'skyl4', 'snowdepth', 'peak_wind_time']

# List of placeholders to replace
placeholders = ['M', 'T', '', 'NaN', 'NULL', 'None']

# Replace placeholders with np.nan
df[continuous_cols] = df[continuous_cols].replace(placeholders, np.nan).astype(str)

# Convert continuous columns to numeric, coercing errors to np.nan
for col in continuous_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

C:\Users\neela\AppData\Local\Temp\ipykernel_29008\2860399940.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[continuous_cols] = df[continuous_cols].replace(placeholders, np.nan).astype(str)


In [18]:
print("Missing values in continuous columns before processing:")
print(df[continuous_cols].isnull().sum())

Missing values in continuous columns before processing:
tmpf                   6865
dwpf                   7015
relh                   7112
feel                   7125
drct                  25052
sknt                   5024
gust                 101714
peak_wind_gust       108345
peak_wind_drct       108345
alti                  12732
mslp                  37235
vsby                   9941
p01i                   7730
ice_accretion_1hr    114664
ice_accretion_3hr    114664
ice_accretion_6hr    114664
skyl1                 54223
skyl2                 93650
skyl3                107296
skyl4                114664
snowdepth            114664
peak_wind_time       114664
dtype: int64


In [19]:
# Step 3: Handle missing values in continuous variables


# Identify columns to drop due to high NaN count
nan_threshold = df.shape[0] / 2                     # Remove columns with more than 50% missing values
print(f"nan thresh is {nan_threshold}")
bad_columns = [col for col in df.columns if df[col].isnull().sum() >= nan_threshold]
print(f"bad columns are {bad_columns}")

# Add less relevant and irrelevant features to the removal list
irrelevant_features = [
    'ice_accretion_1hr', 'ice_accretion_3hr', 'ice_accretion_6hr',  # Fully NaN
    'skyl1', 'skyl2', 'skyl3', 'skyl4',  # Less relevant (Sky level altitudes)
    'skyc1', 'skyc2', 'skyc3', 'skyc4',  # Less relevant (Sky coverage)
    'wxcodes',  # Categorical, redundant with precipitation/visibility
    'metar'  # Text format, unusable directly
]

# Combine both lists and ensure no duplicates
columns_to_remove = list(set(bad_columns + irrelevant_features))
df.drop(columns=columns_to_remove, inplace=True)

# Update continuous columns to exclude removed columns
continuous_cols = list(set(continuous_cols) - set(columns_to_remove))
print(f"Remaining continuous columns: {continuous_cols}")


# Apply linear interpolation within each station group using transform
df[continuous_cols] = df.groupby('station')[continuous_cols].transform(
    lambda group: group.interpolate(method='linear')
)

# Handle any remaining missing values with forward and backward fill using transform
df[continuous_cols] = df.groupby('station')[continuous_cols].transform(
    lambda group: group.ffill().bfill()
)


# Verify missing values are filled
print("Missing values in continuous columns after processing:")
print(df[continuous_cols].isnull().sum())

nan thresh is 57332.0
bad columns are ['gust', 'skyl2', 'skyl3', 'skyl4', 'ice_accretion_1hr', 'ice_accretion_3hr', 'ice_accretion_6hr', 'peak_wind_gust', 'peak_wind_drct', 'peak_wind_time', 'snowdepth']
Remaining continuous columns: ['feel', 'dwpf', 'p01i', 'relh', 'mslp', 'alti', 'vsby', 'tmpf', 'drct', 'sknt']
Missing values in continuous columns after processing:
feel    0
dwpf    0
p01i    0
relh    0
mslp    0
alti    0
vsby    0
tmpf    0
drct    0
sknt    0
dtype: int64


In [20]:
# Step 5: Feature scaling
# List of all features (excluding 'valid' and 'metar')
feature_cols = continuous_cols #+ categorical_cols
print(df[feature_cols].shape)

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the features
df[feature_cols] = scaler.fit_transform(df[feature_cols])


(114664, 10)


In [21]:
# Step 6: Prepare sequences for LSTM input
# Assuming we are predicting 'tmpf' (temperature) as the target variable
# and using previous 24 time steps/8 hours (n_steps_in) to predict the next time step/20 minutes from now (n_steps_out)
# create sliding window sequences X: (114640, 24, 10), y: (114640, 10)

n_steps_in = 24  # Number of past time steps
n_steps_out = 1  # Number of future time steps to predict

# We'll create sequences for each station separately
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data) - n_steps_in - n_steps_out + 1):
        X.append(data[i:(i + n_steps_in), :])
        y.append(data[(i + n_steps_in):(i + n_steps_in + n_steps_out), :])
    return np.array(X), np.array(y)

# Prepare data for each station
X_list = []
y_list = []
stations = df['station'].unique()

for station in stations:
    station_data = df[df['station'] == station]
    station_data = station_data.reset_index(drop=True)
    data_values = station_data[feature_cols].values
    # target_col_index = feature_cols.index('tmpf')  # Index of target variable in features

    X_station, y_station = create_sequences(data_values, n_steps_in, n_steps_out)
    X_list.append(X_station)
    y_list.append(y_station)


# Concatenate data from all stations
X = np.concatenate(X_list, axis=0)
y = np.concatenate(y_list, axis=0)


if n_steps_out == 1:
    y = y.squeeze(1)  # Shape becomes (num_samples, num_features) = (114640, 10) for JRB


print(X.shape)
print(y.shape)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

(114640, 24, 10)
(114640, 10)


In [22]:
# Step 7: Split the data into training and testing sets
# Since it's time-series data, we'll use the first 80% for training and the rest for testing
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Now the data is ready for training the LSTM model

# Define a PyTorch Dataset
class WeatherDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [30]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

# Create DataLoaders
batch_size = 32
train_dataset = WeatherDataset(X_train, y_train)
test_dataset = WeatherDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Hyperparameters for SegRNN
input_size = X.shape[2]  # Number of features
hidden_size = 512  # Based on the SEGRNN paper
output_size = X.shape[2]  # Predict all features
segment_length = 8  # Based on the SEGRNN paper
learning_rate = 0.001

# Initialize SegRNNModel
model = SegRNNModel(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    segment_length=segment_length,
    learning_rate=learning_rate
)

# Logger
logger = TensorBoardLogger("logs", name="segrnn_experiment")

# Checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/",
    filename="segrnn-{epoch:02d}-{val_loss:.4f}",
    save_top_k=1,
    monitor="val_loss",
    mode="min"
)

# Trainer with logging and checkpointing
trainer = Trainer(
    max_epochs=4,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, train_loader)

# Optional: Evaluate on the test set
trainer.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | model     | SegRNN  | 2.4 M  | train
1 | criterion | MSELoss | 0      | train
----------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.544     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=4` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.12028153985738754    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.12028153985738754}]

In [31]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [32]:
%tensorboard --logdir logs --port=6006


Reusing TensorBoard on port 6006 (pid 23700), started 0:06:17 ago. (Use '!kill 23700' to kill it.)